## Data Downloading and loading

Code below downloads data via Kaggle API if the files are
not already present in the directory (see `/src/utilities/download_data.sh`).
Additionally sets up necessary global variables for this notebook:

In [1]:
import pathlib
import pandas as pd

# Where data will be stored if not present
DATA_PATH = "../input"
! ./utilities/download_data.sh "$DATA_PATH"

PREPROCESSING_PATH = pathlib.Path("../preprocessing")

# Where models are/will be stored
PREDICTIONS_PATH = pathlib.Path("../predictions")
PREDICTIONS_PATH.mkdir(parents=True, exist_ok=True)

test_data.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_data.csv.zip: Skipping, found more recently modified local copy (use --force to force download)
train_labels.csv: Skipping, found more recently modified local copy (use --force to force download)
sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
/home/vyz/projects/Kaggle1NN2019/src
Script ran successfully

Data loaded below has been constructed in `preprocessing` notebook:

In [2]:
from utilities.general import train_data, test_data

_, y = train_data(pathlib.Path(DATA_PATH))
X_test = test_data(pathlib.Path(DATA_PATH))

X_train=pd.read_csv(PREPROCESSING_PATH / pathlib.Path("algorithmic_data.csv"), index_col=0)
X_test = X_test[X_train.columns]

variance_dataset=pd.read_csv(PREPROCESSING_PATH / pathlib.Path("variance_data.csv"), index_col=0)

Settings below were found to achieve highest score on public leaderboard.

Various were tried (more ensemble datasets, less, more models, different seed each time), those tries are not documented in this repository (though they are saved if anyone wants to see).

In [3]:
import numpy as np

HOW_MANY_MODELS = 80
HOW_MANY_DATASETS = 1

MAIN_SEED = 42

## Create N models

Using utilities we may easily create many different configurations of models.

`create_configs` will create `HOW_MANY_MODELS` configurations of models (those configurations are `input` and `output` independent, hence can be used for varying amount of features (see: `HOW_MANY_DATASETS`).

For information about how those networks are created, consult `utilities.generator.create_configs` function.

In [4]:
from utilities.training import generate_configs

model_configs = generate_configs(
    max_layers=5,
    max_width=900,
    min_width=100,
    how_many=HOW_MANY_MODELS,
    seed=MAIN_SEED,
)

`ensemble_datasets` creates generator of datasets with varying count of features.

`split_feature` indicates how many best features (as indicated by analysis) will be used for "ensemble dataset". Additionally 96 - `split_feature` will be randomly chosen (say `25` features were chosen from the less important features in range `30-96` if we were to follow the example below):

In [5]:
from utilities.training import ensemble_datasets

datasets = ensemble_datasets(
    X_train,
    X_test,
    split_feature=30,
    how_many_models=HOW_MANY_MODELS,
    how_many_datasets=HOW_MANY_DATASETS,
    seed=MAIN_SEED,
)

Once configurations of random models are created (see appropriate functions in utils) and dataset generators setup we can train and predict using third part library [`skorch`](https://github.com/skorch-dev/skorch) which removes mental load nicely.

Characteristics of training loop applied to every generated neural network:

  - Train neural network with maximum of 40 epochs
  - Optimizer used: `Adam` with default learning rate
  - Batch size equal to 64
  - Stratified Validation set being 20% of the train data
  - Train data shuffled after each epoch
  - Early Stopping if no validation accuracy improvement after `8` epochs
  - Learning rate multiplied by `0.6` if no validation accuracy improvement after 2 epochs

Additionally, no models are saved but make a prediction on test set using state achieved as the best in validation accuracy.

Logits of predictions are saved for each model with it's descriptive name and validation accuracy achieved (see `prediction` data) so we can try different ensembling techniques if we so wish (e.g. thresholding on accuracy and using only the best models).

In [6]:
from utilities.training import predict_with_models

predict_with_models(
    PREDICTIONS_PATH,
    y,
    X_test,
    model_configs,
    datasets,
    10,
    MAIN_SEED,
)

Current best validation, making test prediction
  epoch    train_loss    valid_loss    validation_accuracy     dur
-------  ------------  ------------  ---------------------  ------
      1        0.6802        0.5876                 0.7774  7.4093
Current best validation, making test prediction
      2        0.5529        0.5225                 0.7979  5.6897
Current best validation, making test prediction
      3        0.5098        0.5210                 0.8033  5.5162
Current best validation, making test prediction
      4        0.4793        0.4585                 0.8235  4.9231
Current best validation, making test prediction
      5        0.4582        0.4558                 0.8291  5.6262
      6        0.4372        0.4701                 0.8196  5.0378
      7        0.4192        0.4601                 0.8265  5.3469
Current best validation, making test prediction
      8        0.4024        0.4426                 0.8357  5.3459
      9        0.3889        0.4532       

Stopping since validation_accuracy has not improved in the last 8 epochs.
0.8546492111446794
Saving predictions in ../predictions/0.8546492111446794_0_Layers=[869, 869, 869, 869],Batchnorm,Activation=ReLU.csv
Current best validation, making test prediction
  epoch    train_loss    valid_loss    validation_accuracy     dur
-------  ------------  ------------  ---------------------  ------
      1        0.6008        0.5116                 0.8000  7.9101
Current best validation, making test prediction
      2        0.4800        0.4491                 0.8288  7.5377
      3        0.4314        0.4604                 0.8269  7.3128
Current best validation, making test prediction
      4        0.3943        0.4192                 0.8411  6.8846
Current best validation, making test prediction
      5        0.3644        0.4334                 0.8458  4.3207
      6        0.3424        0.4275                 0.8443  4.5623
      7        0.3153        0.4441                 0.8347  5.9

Current best validation, making test prediction
     13        0.3468        0.4557                 0.8395  3.3737
     14        0.3375        0.4836                 0.8345  3.9352
     15        0.3280        0.4867                 0.8357  3.9125
     16        0.3222        0.4722                 0.8394  4.0333
Epoch    16: reducing learning rate of group 0 to 6.0000e-04.
Current best validation, making test prediction
     17        0.2756        0.4540                 0.8442  4.1669
Current best validation, making test prediction
     18        0.2653        0.4547                 0.8464  3.3019
     19        0.2576        0.4667                 0.8422  4.0161
     20        0.2507        0.4718                 0.8436  4.0608
     21        0.2438        0.4884                 0.8395  4.0491
Epoch    21: reducing learning rate of group 0 to 3.6000e-04.
Current best validation, making test prediction
     22        0.2171        0.4610                 0.8505  3.9274
     23       

     17        0.0363        1.0047                 0.8513  4.8450
Epoch    17: reducing learning rate of group 0 to 3.6000e-04.
     18        0.0149        1.0359                 0.8520  4.6645
Current best validation, making test prediction
     19        0.0079        1.1403                 0.8538  4.6736
     20        0.0086        1.1815                 0.8512  4.8061
     21        0.0113        1.1939                 0.8537  4.7502
     22        0.0145        1.2033                 0.8500  4.7950
Epoch    22: reducing learning rate of group 0 to 2.1600e-04.
     23        0.0046        1.2458                 0.8535  4.7341
Current best validation, making test prediction
     24        0.0009        1.3063                 0.8560  4.3728
Current best validation, making test prediction
     25        0.0004        1.3731                 0.8561  4.0033
     26        0.0002        1.4344                 0.8553  4.5621
     27        0.0001        1.4893                 0.8560  3.

     25        0.1699        0.4891                 0.8504  3.8813
     26        0.1668        0.4911                 0.8504  3.7455
     27        0.1627        0.4967                 0.8491  3.2811
Epoch    27: reducing learning rate of group 0 to 1.2960e-04.
     28        0.1529        0.4985                 0.8507  3.9473
     29        0.1502        0.5024                 0.8517  4.0516
     30        0.1474        0.5034                 0.8495  3.8373
Epoch    30: reducing learning rate of group 0 to 7.7760e-05.
     31        0.1417        0.5060                 0.8494  4.0630
Stopping since validation_accuracy has not improved in the last 8 epochs.
0.8528029540114133
Saving predictions in ../predictions/0.8528029540114133_0_Layers=[341, 341],Activation=SELU.csv
Current best validation, making test prediction
  epoch    train_loss    valid_loss    validation_accuracy     dur
-------  ------------  ------------  ---------------------  ------
      1        0.6881        0.5968 

...

Current best validation, making test prediction
      2        0.5402        0.5175                 0.8026  4.3603
Current best validation, making test prediction
      3        0.4891        0.4810                 0.8182  4.1627
Current best validation, making test prediction
      4        0.4625        0.4731                 0.8241  4.5553
Current best validation, making test prediction
      5        0.4364        0.4588                 0.8281  4.3679
Current best validation, making test prediction
      6        0.4145        0.4453                 0.8321  3.9356
Current best validation, making test prediction
      7        0.3977        0.4445                 0.8344  4.7269
Current best validation, making test prediction
      8        0.3826        0.4378                 0.8356  3.4714
      9        0.3651        0.4397                 0.8354  4.2378
Current best validation, making test prediction
     10        0.3518        0.4329                 0.8431  4.1716
Current best 

     15        0.1990        0.5252                 0.8452  3.9088
     16        0.1853        0.5618                 0.8379  3.3465
     17        0.1707        0.5874                 0.8468  3.3995
Epoch    17: reducing learning rate of group 0 to 3.6000e-04.
     18        0.1216        0.5840                 0.8478  3.0948
Current best validation, making test prediction
     19        0.1030        0.5992                 0.8489  4.2925
     20        0.0925        0.6473                 0.8459  4.0530
     21        0.0844        0.6897                 0.8432  3.9999
     22        0.0742        0.7210                 0.8464  2.4604
Epoch    22: reducing learning rate of group 0 to 2.1600e-04.
     23        0.0501        0.7229                 0.8484  4.0173
     24        0.0411        0.7540                 0.8489  2.9395
     25        0.0362        0.7809                 0.8456  3.1530
Epoch    25: reducing learning rate of group 0 to 1.2960e-04.
     26        0.0260        

Current best validation, making test prediction
      5        0.4549        0.4486                 0.8284  5.3750
Current best validation, making test prediction
      6        0.4298        0.4302                 0.8384  7.2278
      7        0.4147        0.4259                 0.8366  7.2891
      8        0.3952        0.4397                 0.8343  5.6236
Current best validation, making test prediction
      9        0.3797        0.4224                 0.8419  6.6514
     10        0.3636        0.4496                 0.8306  5.1183
Current best validation, making test prediction
     11        0.3524        0.4244                 0.8429  5.7897
Current best validation, making test prediction
     12        0.3389        0.4135                 0.8463  5.7401
     13        0.3265        0.4257                 0.8441  5.0752
Current best validation, making test prediction
     14        0.3130        0.4197                 0.8475  4.8117
     15        0.2979        0.4255       

Current best validation, making test prediction
      9        0.4220        0.4746                 0.8246  4.9744
Current best validation, making test prediction
     10        0.4123        0.4621                 0.8254  3.4829
     11        0.4014        0.4754                 0.8218  5.3965
Current best validation, making test prediction
     12        0.3900        0.4571                 0.8313  5.3640
     13        0.3816        0.4743                 0.8249  5.1570
Current best validation, making test prediction
     14        0.3697        0.4568                 0.8320  5.3808
     15        0.3608        0.4650                 0.8314  5.8588
Current best validation, making test prediction
     16        0.3523        0.4493                 0.8345  5.3837
     17        0.3438        0.4598                 0.8292  5.9424
Current best validation, making test prediction
     18        0.3354        0.4677                 0.8353  5.2660
     19        0.3273        0.4790       

Current best validation, making test prediction
      4        0.5024        0.4922                 0.8113  4.1462
Current best validation, making test prediction
      5        0.4799        0.4705                 0.8180  6.0030
Current best validation, making test prediction
      6        0.4560        0.4726                 0.8208  3.7645
Current best validation, making test prediction
      7        0.4384        0.4441                 0.8315  5.5632
      8        0.4294        0.4430                 0.8304  5.5944
Current best validation, making test prediction
      9        0.4168        0.4346                 0.8367  4.4495
     10        0.4028        0.4283                 0.8364  4.0771
     11        0.3952        0.4324                 0.8367  3.8740
Current best validation, making test prediction
     12        0.3847        0.4290                 0.8402  4.4046
Current best validation, making test prediction
     13        0.3768        0.4209                 0.8426  6